<a href="https://colab.research.google.com/github/Shriansh16/NLP/blob/main/Fake_News_Classification_using_LSTM_and_several_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [1]:
import pandas as pd
import os

In [2]:
!unzip /content/train_LSTM.zip

Archive:  /content/train_LSTM.zip
  inflating: train_LSTM.csv          


In [3]:
df=pd.read_csv('/content/train_LSTM.csv')

In [24]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [25]:
df.shape

(20800, 5)

In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df=df.dropna()

In [6]:
df.reset_index(drop=True,inplace=True)

In [29]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
X=df.drop('label',axis=1)
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
X.drop('id',axis=1,inplace=True)

In [8]:
import tensorflow as tf


In [9]:
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
messages1=X.copy()

## DATA CLEANING


In [ ]:
def tolower(text):
  return text.lower()

In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(tolower)

In [ ]:
messages1.head()

,title,author,text
0,house dem aide: we didn’t even see comey’s let...,darrell lucus,house dem aide: we didn’t even see comey’s let...
1,"flynn: hillary clinton, big woman on campus - ...",daniel j. flynn,ever get the feeling your life circles the rou...
2,why the truth might get you fired,consortiumnews.com,"why the truth might get you fired october 29, ..."
3,15 civilians killed in single us airstrike hav...,jessica purkiss,videos 15 civilians killed in single us airstr...
4,iranian woman jailed for fictional unpublished...,howard portnoy,print \nan iranian woman has been sentenced to...


In [ ]:
## REMOVING HTML TAGS
import re
def remove_html_tags(text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',text)

In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_html_tags)

In [ ]:
## removing url
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)

In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_url)

In [ ]:
## HANDLING PUNCTUATION
import string
def remove_punc(text):
  for i in string.punctuation:
    text=text.replace(i,'')
  return text


In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_punc)

In [ ]:
messages1.head()

,title,author,text
0,house dem aide we didn’t even see comey’s lett...,darrell lucus,house dem aide we didn’t even see comey’s lett...
1,flynn hillary clinton big woman on campus bre...,daniel j flynn,ever get the feeling your life circles the rou...
2,why the truth might get you fired,consortiumnewscom,why the truth might get you fired october 29 2...
3,15 civilians killed in single us airstrike hav...,jessica purkiss,videos 15 civilians killed in single us airstr...
4,iranian woman jailed for fictional unpublished...,howard portnoy,print \nan iranian woman has been sentenced to...


In [ ]:
## REMOVING STOPWORDS
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def remove_stopwords(text):
  words=text.split()
  for i,word in enumerate(words):
    if word in stopwords.words('english'):
      words[i]=''
  return ' '.join(words)

In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_stopwords)

In [ ]:
messages1['title'][2]

'  truth might get  fired'

In [ ]:
def remove_extra_spaces(sentence):

    cleaned_sentence = ' '.join(sentence.split()).strip()

    return cleaned_sentence

In [ ]:
for i in messages1.columns:
  messages1[i]=messages1[i].apply(remove_extra_spaces)

In [ ]:
messages1['title'][2]

'truth might get fired'

In [ ]:
messages1.to_csv('cleaned_data.csv')

In [10]:
!unzip /content/cleaned_data.zip

Archive:  /content/cleaned_data.zip
  inflating: cleaned_data.csv        


In [11]:
msg=pd.read_csv('/content/cleaned_data.csv')

In [12]:
msg=msg['title']

In [13]:
msg

0        house dem aide didn’t even see comey’s letter ...
1         flynn hillary clinton big woman campus breitbart
2                                    truth might get fired
3        15 civilians killed single us airstrike identi...
4        iranian woman jailed fictional unpublished sto...
                               ...                        
18280       rapper ti trump ’poster child white supremacy’
18281    nfl playoffs schedule matchups odds new york t...
18282    macy’s said receive takeover approach hudson’s...
18283          nato russia hold parallel exercises balkans
18284                                      keeps f35 alive
Name: title, Length: 18285, dtype: object

In [14]:
sentences=list(msg)

In [15]:
sentences

['house dem aide didn’t even see comey’s letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 '15 civilians killed single us airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathrooms exclusive video breitbart',
 'benoît hamon wins french socialist party’s presidential nomination new york times',
 'backchannel plan ukraine russia courtesy trump associates new york times',
 'obama’s organizing action partners soroslinked ‘indivisible’ disrupt trump’s agenda',
 'bbc comedy sketch real housewives isis causes outrage',
 'russian researchers discover secret nazi military base ‘treasure hunter’ arctic photos',
 'us officials see link trump russia',
 'yes paid government trolls social media blogs forums websites',
 'major league soccer argentines find home success new york times',
 'wells fargo chief abruptly 

In [16]:
sentences = [str(sentence) for sentence in sentences]

In [17]:
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 5000   #vocabulary size
onehot_repe = [one_hot(i,voc_size)for i in sentences]      #The one_hot function will encode words into integers, and the integers will range from 0 to voc_size - 1.
onehot_repe

[[521, 1146, 2909, 1703, 839, 3359, 4007, 607, 3196, 2739, 4229],
 [3954, 1164, 2159, 4849, 4262, 2331, 4376],
 [1269, 4593, 4779, 4462],
 [3173, 3427, 1260, 4386, 51, 279, 3256],
 [2615, 4262, 3729, 2740, 3098, 2204, 4262, 1927, 965, 1170],
 [1651,
  4726,
  379,
  4842,
  623,
  4684,
  2751,
  2084,
  2524,
  759,
  4317,
  3450,
  3620,
  3988,
  4376],
 [2114, 1034, 2329, 298, 1727, 2065, 4663, 88, 4196, 3064, 3422],
 [1178, 1454, 1009, 4446, 770, 4684, 3510, 4196, 3064, 3422],
 [4515, 3124, 2071, 1366, 1132, 2767, 1899, 4324, 3242],
 [1344, 740, 3640, 182, 1766, 686, 3014, 4028],
 [1921, 1993, 1691, 1242, 393, 534, 963, 2121, 147, 2940, 3556],
 [51, 3030, 3359, 2622, 4684, 4446],
 [763, 3499, 1057, 4522, 479, 1183, 1158, 4215, 2606],
 [1843, 3693, 4697, 3386, 1826, 3992, 1141, 4196, 3064, 3422],
 [1540, 1122, 725, 77, 778, 4196, 3064, 3422],
 [4108, 1066, 2235, 935, 2251, 1930, 541, 379, 911, 2149, 3911],
 [2676, 2127, 1164],
 [4405, 4431, 3763, 1712, 4684, 4617, 2553, 4376],
 [4

In [18]:
onehot_repe[0]

[521, 1146, 2909, 1703, 839, 3359, 4007, 607, 3196, 2739, 4229]

In [19]:
sentences[0]

'house dem aide didn’t even see comey’s letter jason chaffetz tweeted'

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sent_length = 50
embedded_doc = pad_sequences(onehot_repe, maxlen = sent_length, padding = 'pre')

In [21]:
embedded_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,  521, 1146, 2909, 1703,  839,
       3359, 4007,  607, 3196, 2739, 4229], dtype=int32)

## CREATING MODEL

In [22]:
embedding_vector_features = 50

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics= ['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            250000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 310501 (1.18 MB)
Trainable params: 310501 (1.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
from sklearn.model_selection import train_test_split
import numpy as np

In [24]:
X_input=np.array(embedded_doc)
y_input=np.array(y)

In [25]:
X_input.shape

(18285, 50)

In [26]:
y_input.shape

(18285,)

In [27]:
X_train,X_test,y_train,y_test=train_test_split(X_input,y_input,test_size=0.33,random_state=42)

In [37]:
# training
model.fit(X_train,y_train,epochs=5,batch_size=64,validation_data=(X_test,y_test))

Epoch 1/5
192/192 [==============================] - 20s 90ms/step - loss: 0.3219 - accuracy: 0.8461 - val_loss: 0.1975 - val_accuracy: 0.9176
Epoch 2/5
192/192 [==============================] - 17s 90ms/step - loss: 0.1251 - accuracy: 0.9510 - val_loss: 0.2186 - val_accuracy: 0.9100
Epoch 3/5
192/192 [==============================] - 17s 90ms/step - loss: 0.0715 - accuracy: 0.9746 - val_loss: 0.2320 - val_accuracy: 0.9122
Epoch 4/5
192/192 [==============================] - 18s 91ms/step - loss: 0.0370 - accuracy: 0.9882 - val_loss: 0.3294 - val_accuracy: 0.9133
Epoch 5/5
192/192 [==============================] - 16s 85ms/step - loss: 0.0169 - accuracy: 0.9956 - val_loss: 0.4237 - val_accuracy: 0.9130


In [38]:
# lets check how it is performing
y_pred = (model.predict(X_test)>0.5).astype("int64")
'''
After making predictions, this part of the code compares each prediction with 0.5.
The result is a Boolean array where each element is True if the corresponding prediction is greater than 0.5 and False otherwise.
Finally, the code converts the Boolean array obtained from the previous step to an array of integers (int64).
'''

189/189 [==============================] - 4s 15ms/step


'\nAfter making predictions, this part of the code compares each prediction with 0.5.\nThe result is a Boolean array where each element is True if the corresponding prediction is greater than 0.5 and False otherwise.\nFinally, the code converts the Boolean array obtained from the previous step to an array of integers (int64).\n'

In [39]:
y_pred.shape

(6035, 1)

In [40]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [41]:
confusion_matrix(y_test,y_pred)

array([[3126,  293],
       [ 232, 2384]])

In [42]:
accuracy_score(y_test,y_pred)

0.9130074565037283

## BIDIRECTIONAL LSTM

In [28]:
#create model
embedding_vector_features = 50

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

model1 = Sequential()
model1.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1,activation = 'sigmoid'))

model1.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics= ['accuracy'])
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 50)            250000    
                                                                 
 bidirectional (Bidirection  (None, 200)               120800    
 al)                                                             
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 371001 (1.42 MB)
Trainable params: 371001 (1.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
model1.fit(X_train,y_train, epochs = 5, batch_size = 64, validation_data = (X_test,y_test))

Epoch 1/5
192/192 [==============================] - 39s 172ms/step - loss: 0.3244 - accuracy: 0.8406 - val_loss: 0.1957 - val_accuracy: 0.9158
Epoch 2/5
192/192 [==============================] - 33s 174ms/step - loss: 0.1184 - accuracy: 0.9561 - val_loss: 0.2186 - val_accuracy: 0.9167
Epoch 3/5
192/192 [==============================] - 37s 194ms/step - loss: 0.0651 - accuracy: 0.9782 - val_loss: 0.2786 - val_accuracy: 0.9110
Epoch 4/5
192/192 [==============================] - 33s 170ms/step - loss: 0.0365 - accuracy: 0.9882 - val_loss: 0.3238 - val_accuracy: 0.9074
Epoch 5/5
192/192 [==============================] - 34s 175ms/step - loss: 0.0190 - accuracy: 0.9949 - val_loss: 0.4175 - val_accuracy: 0.9132


In [30]:
# Prediction
y_pred = (model1.predict(X_test)>0.5).astype("int64")
y_pred.shape

189/189 [==============================] - 7s 28ms/step


(6035, 1)

In [31]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3046,  373],
       [ 151, 2465]])

In [32]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9131731565865783

## RNN

In [33]:
#create model
embedding_vector_features = 50

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, SimpleRNN

model2 = Sequential()
model2.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model2.add(SimpleRNN(100))
model2.add(Dense(1,activation = 'sigmoid'))

model2.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics= ['accuracy'])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            250000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               15100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 265201 (1.01 MB)
Trainable params: 265201 (1.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
model2.fit(X_train,y_train, epochs = 2, batch_size = 64, validation_data = (X_test,y_test))

Epoch 1/2
192/192 [==============================] - 9s 35ms/step - loss: 0.2977 - accuracy: 0.8692 - val_loss: 0.1930 - val_accuracy: 0.9122
Epoch 2/2
192/192 [==============================] - 6s 29ms/step - loss: 0.0783 - accuracy: 0.9727 - val_loss: 0.2229 - val_accuracy: 0.9148


In [36]:
# Prediction
y_pred = (model2.predict(X_test)>0.5).astype("int64")
y_pred.shape

189/189 [==============================] - 3s 13ms/step


(6035, 1)

In [37]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3100,  319],
       [ 195, 2421]])

In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9148301574150787

## BI- DIRECTIONAL RNN

In [39]:
#create model
embedding_vector_features = 50

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, SimpleRNN

model3 = Sequential()
model3.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model3.add(Bidirectional(SimpleRNN(100)))
model3.add(Dense(1,activation = 'sigmoid'))

model3.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics= ['accuracy'])
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 50, 50)            250000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               30200     
 onal)                                                           
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 280401 (1.07 MB)
Trainable params: 280401 (1.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
model3.fit(X_train,y_train, epochs = 2, batch_size = 64, validation_data = (X_test,y_test))

Epoch 1/2
192/192 [==============================] - 14s 49ms/step - loss: 0.2832 - accuracy: 0.8869 - val_loss: 0.2993 - val_accuracy: 0.8651
Epoch 2/2
192/192 [==============================] - 11s 58ms/step - loss: 0.1892 - accuracy: 0.9238 - val_loss: 0.2167 - val_accuracy: 0.9064


In [41]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)


array([[3100,  319],
       [ 195, 2421]])

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9148301574150787

In [43]:
#Comparing different confusion matrix using type 1 and type 2 error
'''
LSTM
array([[3119,  300],
       [ 219, 2397]])

bi-LSTM
array([[3115,  304],
       [ 231, 2385]])

RNN
array([[2999,  420],
       [  84, 2532]])


bi-RNN
array([[3129,  290],
       [ 238, 2378]])
'''

'\nLSTM\narray([[3119,  300],\n       [ 219, 2397]])\n\nbi-LSTM\narray([[3115,  304],\n       [ 231, 2385]])\n\nRNN\narray([[2999,  420],\n       [  84, 2532]])\n\n\nbi-RNN\narray([[3129,  290],\n       [ 238, 2378]])\n'

In [44]:
# if there are lots hidden layers then it can lead to overfitting
# hence we will use dropout
# if we use dropout(0.3) then randomly 30 percent of the neurons will be deactivated

In [45]:
#lets use dropout and check its effects

In [46]:
#create model
embedding_vector_features = 50

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model4 = Sequential()
model4.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model4.add(Dropout(0.3))
model4.add(LSTM(100))
model4.add(Dropout(0.3))
model4.add(Dense(1,activation = 'sigmoid'))

model4.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics= ['accuracy'])
model4.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 50, 50)            250000    
                                                                 
 dropout (Dropout)           (None, 50, 50)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               60400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 310501 (1.18 MB)
Trainable params: 310501 (1.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [47]:
model4.fit(X_train,y_train, epochs = 2, batch_size = 64, validation_data = (X_test,y_test))

Epoch 1/2
192/192 [==============================] - 26s 111ms/step - loss: 0.3370 - accuracy: 0.8369 - val_loss: 0.1960 - val_accuracy: 0.9171
Epoch 2/2
192/192 [==============================] - 20s 102ms/step - loss: 0.1309 - accuracy: 0.9491 - val_loss: 0.2007 - val_accuracy: 0.9137


In [48]:
# Prediction
y_pred = (model4.predict(X_test)>0.5).astype("int64")
y_pred.shape

189/189 [==============================] - 5s 24ms/step


(6035, 1)

In [49]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3104,  315],
       [ 206, 2410]])

In [50]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9136702568351284